In [1]:
from itertools import combinations

In [2]:
import pandas as pd
import numpy as np

In [3]:
# import results data for the current season as of Nov 23 2018
results = pd.read_csv('http://www.football-data.co.uk/mmz4281/1819/E0.csv')

In [4]:
# results.to_csv('pl_results_20181123.csv', encoding='utf-8', index=False)

In [5]:
results.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,E0,10/08/18,Man United,Leicester,2,1,H,1,0,H,...,1.79,17,-0.75,1.75,1.70,2.29,2.21,1.55,4.07,7.69
1,E0,11/08/18,Bournemouth,Cardiff,2,0,H,1,0,H,...,1.83,20,-0.75,2.20,2.13,1.80,1.75,1.88,3.61,4.70
2,E0,11/08/18,Fulham,Crystal Palace,0,2,A,0,1,A,...,1.87,22,-0.25,2.18,2.11,1.81,1.77,2.62,3.38,2.90
3,E0,11/08/18,Huddersfield,Chelsea,0,3,A,0,2,A,...,1.84,23,1.00,1.84,1.80,2.13,2.06,7.24,3.95,1.58
4,E0,11/08/18,Newcastle,Tottenham,1,2,A,1,2,A,...,1.81,20,0.25,2.20,2.12,1.80,1.76,4.74,3.53,1.89


In [6]:
# create modified results dataframe in required format
home_results = (results
               .assign(points = results.FTR.map({'H': 3, 'A': 0, 'D': 1}),
                       venue = 'home')
               .rename(columns = {
                   'HomeTeam': 'team',
                   'AwayTeam': 'opponent'
               })
               .loc[:, ['team','opponent','venue','points']])

away_results = (results
               .assign(points = results.FTR.map({'A': 3, 'H': 0, 'D': 1}),
                       venue = 'away')
               .rename(columns = {
                   'AwayTeam': 'team',
                   'HomeTeam': 'opponent'
               })
               .loc[:, ['team','opponent','venue','points']])

modified_results = pd.concat([home_results, away_results], ignore_index=True, sort=False)

In [7]:
modified_results.head()

,team,opponent,venue,points
0,Man United,Leicester,home,3
1,Bournemouth,Cardiff,home,3
2,Fulham,Crystal Palace,home,0
3,Huddersfield,Chelsea,home,0
4,Newcastle,Tottenham,home,0


In [8]:
teams = pd.unique(results.HomeTeam)
teams.sort()

In [9]:
teams

array(['Arsenal', 'Bournemouth', 'Brighton', 'Burnley', 'Cardiff',
       'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Huddersfield',
       'Leicester', 'Liverpool', 'Man City', 'Man United', 'Newcastle',
       'Southampton', 'Tottenham', 'Watford', 'West Ham', 'Wolves'],
      dtype=object)

In [10]:
# create the W matrix
mapping_table = pd.DataFrame(index=list(combinations(teams, 2)), columns=teams).fillna(0)

In [11]:
for idx, row in mapping_table.iterrows():
    mapping_table.loc[idx, idx[0]] = 1
    mapping_table.loc[idx, idx[1]] = -1

In [12]:
mapping_table.head()

,Arsenal,Bournemouth,Brighton,Burnley,Cardiff,Chelsea,Crystal Palace,Everton,Fulham,Huddersfield,Leicester,Liverpool,Man City,Man United,Newcastle,Southampton,Tottenham,Watford,West Ham,Wolves
"(Arsenal, Bournemouth)",1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"(Arsenal, Brighton)",1,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"(Arsenal, Burnley)",1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"(Arsenal, Cardiff)",1,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"(Arsenal, Chelsea)",1,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
def calculate_points_per_game_difference(teams):
    """Returns points per game difference for teams passed as input."""
    common_games = (modified_results[modified_results.team == teams[0]]
                    .merge(modified_results[modified_results.team == teams[1]],
                           how='inner',
                           on=['opponent','venue']))
    
    if not common_games.empty:
        points_per_game_difference = ((common_games.points_x.sum()
                                       - common_games.points_y.sum())
                                      / common_games.shape[0])
    else:
        points_per_game_difference = 0
    
    return points_per_game_difference

In [14]:
# convert mapping table to numpy array to create 'W'
W = mapping_table.values

In [15]:
# create 'r' using the function defined above
r = (mapping_table
     .index
     .map(calculate_points_per_game_difference)
     .values
     .reshape(mapping_table.shape[0], 1))

In [16]:
# minimizing sum of squared errors to get x
x = np.matmul(np.matmul(np.linalg.inv(np.matmul(W.T, W)), W.T), r)

In [17]:
# unscaled x values for all clubs
sorted(zip(teams, x), key=lambda x: x[1], reverse=True)

[('Man City', array([1.57712054])),
 ('Liverpool', array([1.5531622])),
 ('Arsenal', array([1.48629092])),
 ('Tottenham', array([1.39926525])),
 ('Everton', array([1.13319382])),
 ('Man United', array([1.06506696])),
 ('Bournemouth', array([0.85704985])),
 ('Brighton', array([0.68953683])),
 ('Wolves', array([0.67874814])),
 ('Watford', array([0.65374814])),
 ('Leicester', array([0.63668155])),
 ('Chelsea', array([0.60424107])),
 ('West Ham', array([0.22875744])),
 ('Crystal Palace', array([0.07105655])),
 ('Huddersfield', array([-0.09254092])),
 ('Newcastle', array([-0.18988095])),
 ('Cardiff', array([-0.19429874])),
 ('Southampton', array([-0.22769717])),
 ('Burnley', array([-0.3890997])),
 ('Fulham', array([-0.65875186]))]

In [18]:
x.sum()

10.881649925595237

In [19]:
modified_results.points.mean()

1.3875

In [20]:
# multiplier used for scaling up equals number of matches played by team
m = 12

In [23]:
# add average points per team per game to x and multiply by number of matches
adjusted_points = m * (x + modified_results.points.mean())

In [24]:
sorted(zip(teams, adjusted_points), key=lambda x: x[1], reverse=True)

[('Man City', array([35.57544643])),
 ('Liverpool', array([35.28794643])),
 ('Arsenal', array([34.48549107])),
 ('Tottenham', array([33.44118304])),
 ('Everton', array([30.24832589])),
 ('Man United', array([29.43080357])),
 ('Bournemouth', array([26.93459821])),
 ('Brighton', array([24.92444196])),
 ('Wolves', array([24.79497768])),
 ('Watford', array([24.49497768])),
 ('Leicester', array([24.29017857])),
 ('Chelsea', array([23.90089286])),
 ('West Ham', array([19.39508929])),
 ('Crystal Palace', array([17.50267857])),
 ('Huddersfield', array([15.53950893])),
 ('Newcastle', array([14.37142857])),
 ('Cardiff', array([14.31841518])),
 ('Southampton', array([13.91763393])),
 ('Burnley', array([11.98080357])),
 ('Fulham', array([8.74497768]))]